In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

%matplotlib inline
tf.config.set_visible_devices([], 'GPU')

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder, image_size=(150, 150)):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, image_size)
            img_data = np.asarray(img)
            images.append(img_data)
    return images

In [3]:
def load_images_from_folder_2(folder, image_size=(150, 150)):
    images = []
    for filename in os.listdir(folder):
        loaded_img = load_img(os.path.join(folder,filename), target_size=image_size)
        numpy_img = img_to_array(loaded_img)
        images.append(numpy_img)
    return images

In [4]:
# input_image_dim = (150, 150, 3)
# img_size = (150, 150)
# covid_folder = 'CT_NonCOVID'
# non_covid_folder = 'CT_COVID'

input_image_dim = (250, 250, 3)
img_size = (250, 250)
covid_folder = 'XRAY_PNEUMONIA'
non_covid_folder = 'XRAY_NORMAL'

In [5]:
control_images = load_images_from_folder_2(covid_folder, image_size=img_size)
covid_images = load_images_from_folder_2(non_covid_folder, image_size=img_size)
all_images = control_images + covid_images
all_images = np.array(all_images)
# all_images = all_images.reshape((-1, 150, 150, 3))

In [6]:

all_images.shape

(5856, 250, 250, 3)

In [7]:
# for i in range(4):
#     plt.subplot(2, 2, i + 1)
#     plt.imshow(np.uint8(all_images[i]))
# plt.show

In [8]:
control_labels = [0] * len(control_images)
covid_labels = [1] * len(covid_images)
label_series = pd.Series(control_labels + covid_labels)
print(len(all_images))
print(len(label_series))

5856
5856


In [9]:
X_train, X_test, y_train, y_test = train_test_split(all_images, label_series, test_size=0.25, random_state=42)

In [10]:
print(len(X_train))
print(len(y_train))

4392
4392


In [11]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=input_image_dim)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train, batch_size=1, epochs=3, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'Simple network 1'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 4392 samples
Epoch 1/3
4392/4392 [==============================] - 1090s 248ms/sample - loss: 0.9664 - accuracy: 0.8670
Epoch 2/3
4392/4392 [==============================] - 1096s 249ms/sample - loss: 0.2928 - accuracy: 0.9092
Epoch 3/3
4392/4392 [==============================] - 1093s 249ms/sample - loss: 0.2642 - accuracy: 0.9310
Simple network 1 - accuracy: 0.93784153
Simple network 1 - recall: 0.8746867
Simple network 1 - precision: 0.8948718
Simple network 1 - F1 score: 0.8846641218226838


In [13]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=input_image_dim)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, batch_size=1, epochs=3, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'Simple network 2'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 4392 samples
Epoch 1/3
4392/4392 [==============================] - 1394s 317ms/sample - loss: 0.3614 - accuracy: 0.8848
Epoch 2/3
4392/4392 [==============================] - 1391s 317ms/sample - loss: 0.2011 - accuracy: 0.9340
Epoch 3/3
4392/4392 [==============================] - 1389s 316ms/sample - loss: 0.1546 - accuracy: 0.9449
Simple network 2 - accuracy: 0.95423496
Simple network 2 - recall: 0.9047619
Simple network 2 - precision: 0.925641
Simple network 2 - F1 score: 0.9150824004274438


In [15]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

base_model = tf.keras.applications.MobileNetV2(input_shape=input_image_dim, include_top=False, weights='imagenet')
base_model.trainable = True

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.3)
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    dropout_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\bruce\miniconda3\envs\capstone\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [16]:
model.fit(X_train, y_train, batch_size=1, epochs=2, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'Mobile Net V2'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 4392 samples
Epoch 1/2
4392/4392 [==============================] - 450s 102ms/sample - loss: 0.3018 - accuracy: 0.8704
Epoch 2/2
4392/4392 [==============================] - 447s 102ms/sample - loss: 0.0873 - accuracy: 0.9684
Mobile Net V2 - accuracy: 0.897541
Mobile Net V2 - recall: 0.7593985
Mobile Net V2 - precision: 0.8487395
Mobile Net V2 - F1 score: 0.8015873192370074


In [17]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

base_model = tf.keras.applications.ResNet50V2(input_shape=input_image_dim, include_top=False, weights='imagenet')
base_model.trainable = True

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, batch_size=1, epochs=1, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'ResNet50 V2'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 4392 samples
4392/4392 [==============================] - 1089s 248ms/sample - loss: 0.1833 - accuracy: 0.9303
ResNet50 V2 - accuracy: 0.9474044
ResNet50 V2 - recall: 0.8847118
ResNet50 V2 - precision: 0.9192708
ResNet50 V2 - F1 score: 0.9016603093272982


In [19]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

base_model = tf.keras.applications.VGG16(input_shape=input_image_dim, include_top=False, weights='imagenet')
base_model.trainable = True

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.3)
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dropout_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, batch_size=1, epochs=1, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'VGG16'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 4392 samples
4392/4392 [==============================] - 1833s 417ms/sample - loss: 0.2091 - accuracy: 0.9242
VGG16 - accuracy: 0.91530055
VGG16 - recall: 0.9699248
VGG16 - precision: 0.7755511
VGG16 - F1 score: 0.8619153533373223


In [21]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

base_model = tf.keras.applications.VGG19(input_shape=input_image_dim, include_top=False, weights='imagenet')
base_model.trainable = True

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.3)
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    dropout_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, batch_size=1, epochs=1, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'VGG19'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 4392 samples
4392/4392 [==============================] - 2156s 491ms/sample - loss: 0.2034 - accuracy: 0.9237
VGG19 - accuracy: 0.93784153
VGG19 - recall: 0.9849624
VGG19 - precision: 0.82217574
VGG19 - F1 score: 0.8962371401352061
